In [31]:
from datetime import datetime
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from markdownify import markdownify 
import folium
import io




# endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """
#defaultView:Map

SELECT DISTINCT ?item ?itemLabel ?location ?locationLabel ?latitude ?longitude ?type ?typeLabel ?geo WHERE {


  ?item wdt:P361 wd:Q700216; # Teil von Staatliche Museen zu Berlin
        wdt:P31/wdt:P279* wd:Q33506.
  
  ?item wdt:P625 ?geo.  # Get coordinates
  

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,[AUTO_LANGUAGE]". }

  BIND(geof:latitude(?geo) AS ?latitude).
  BIND(geof:longitude(?geo) AS ?longitude)
}
"""



def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

locations = []

def get_items():

    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else ''
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['placeLabel']['value'] + ', ' + item['countryLabel']['value'] if 'placeLabel' in item else ''
      item_url=item['item']['value'] if 'item' in item else ''
      image_url=item['image']['value'] if 'image' in item else ''

      locations.append((title, latitude, longitude, image_url, item_url, place, alt))






In [32]:
get_items()

m_berlin = folium.Map([52.5097, 13.3675], zoom_start=12)

for name, lat, lon, url, item, place, alt in locations:
    folium.Marker(
        location=[lat, lon],
        popup=f"""<h3><a href='{item}' target='_blank'>{name}</a></h3>
        {place}""",
        tooltip=name  # Hover label
    ).add_to(m_berlin)

print("""
[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flocation%20%3FlocationLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20WHERE%20%7B%0A%0A%0A%20%20%3Fitem%20wdt%3AP361%20wd%3AQ700216%3B%20%23%20Teil%20von%20Staatliche%20Museen%20zu%20Berlin%0A%20%20%20%20%20%20%20%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506.%20%23%20ist%20ein%2Fist%20eine%20Unterklasse%20von%3A%20Museum%0A%20%20%0A%20%20%3Fitem%20wdt%3AP625%20%3Fgeo.%20%20%23%20Get%20coordinates%0A%20%20%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND%28geof%3Alatitude%28%3Fgeo%29%20AS%20%3Flatitude%29.%0A%20%20BIND%28geof%3Alongitude%28%3Fgeo%29%20AS%20%3Flongitude%29%0A%7D%0A)
""")



[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flocation%20%3FlocationLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20WHERE%20%7B%0A%0A%0A%20%20%3Fitem%20wdt%3AP361%20wd%3AQ700216%3B%20%23%20Teil%20von%20Staatliche%20Museen%20zu%20Berlin%0A%20%20%20%20%20%20%20%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506.%20%23%20ist%20ein%2Fist%20eine%20Unterklasse%20von%3A%20Museum%0A%20%20%0A%20%20%3Fitem%20wdt%3AP625%20%3Fgeo.%20%20%23%20Get%20coordinates%0A%20%20%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND%28geof%3Alatitude%28%3Fgeo%29%20AS%20%3Flatitude%29.%0A%20%20BIND%28geof%3Alongitude%28%3Fgeo%29%20AS%20%3Flongitude%29%0A%7D%0A)



In [33]:

m_berlin
